# Deploy <font color='#6957FF'>Widn Tower Anthill</font> Model Package from AWS Marketplace 


---

**Tower Anthill** is designed for advanced translation tasks across multiple languages and domains. It excels at translating complex texts while maintaining context and nuance. The model leverages state-of-the-art language modeling techniques to provide high-quality translations that preserve the original meaning and style of the text. It can handle various content types and specialized terminology across different fields. On our validation dataset, Tower Anthill showed better adaptability than other providers and achieved better COMET scores with few-shot (RAG) examples than standard MT providers. 27 languages are explicitly supported: English, Portuguese, Spanish, French, German, Dutch, Italian, Korean, Chinese, Russian, Japanese, Hindi, Ukrainian, Swedish, Polish, Czech, Romanian, Hungarian, Norwegian, Icelandic, Danish.

This sample notebook shows you how to deploy <font color='#C9FF33'>[Widn Tower Anthill](https://aws.amazon.com/marketplace/pp/prodview-xskn6yectpscq)</font> using Amazon SageMaker.

> **Note**: This model package only supports SageMaker Realtime Inference. Sagemaker doesn't yet support Batch Transform for this type of model package; this is a limitation of the SageMaker Batch Transform service.


## Pre-requisites:
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to <font color='#C9FF33'>[Widn Tower Anthill](https://aws.amazon.com/marketplace/pp/prodview-xskn6yectpscq)</font>. If so, skip step [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

## Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

## Usage instructions
You can run this notebook one cell at a time (by using Shift+Enter for running a cell).

## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='#C9FF33'>[Widn Tower Anthill](https://aws.amazon.com/marketplace/pp/prodview-xskn6yectpscq)</font> in AWS Marketplace.
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [1]:
# Set the region to which you subscribed to the model package
region = "us-east-1"

model_package_arn = f"arn:aws:sagemaker:{region}:571600842260:model-package/widn-tower-3-0-anthill-241001"

If you need to load environment variables from a .env file, you can use the following code:

In [2]:
!pip install python-dotenv
import dotenv

dotenv.load_dotenv()

True

In [3]:
import json

from sagemaker import ModelPackage, Predictor, Session
from sagemaker import deserializers, get_execution_role, serializers

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/kepler/Library/Application Support/sagemaker/config.yaml


In [4]:
session = Session()
region = session.boto_region_name
role = get_execution_role(session)  # Or directly specify a suitable role
role

'arn:aws:iam::571600842260:role/service-role/AmazonSageMaker-ExecutionRole-20241018T132939'

In [5]:
# Choose an instance type to run the model
instance_type = "ml.g5.xlarge"

In [6]:
model_name = model_package_arn.split("/")[-1]

# Create a deployable model from the model package
model = ModelPackage(
    role=role,
    model_package_arn=model_package_arn,
    sagemaker_session=session
)

print(f"{model_name=}")

model_name='widn-tower-3-0-anthill-241001'


## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, [see their documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Create an endpoint

In [ ]:
# Deploy the model
model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    endpoint_name=model_name,
    container_startup_health_check_timeout=500,  # Change if necessary; about 8 minutes is usually enough
)

Once endpoint has been created, you should be able to perform real-time inference.

### B. Create input payload

In [ ]:
# Load the input samples from a jsonlines file in data/input/real-time/source_sentences.jsonl
file_name = "data/input/real-time/source_sentences.jsonl"

with open(file_name, "r") as f:
    inputs = [json.loads(line) for line in f]
inputs

In [ ]:
# Use the following template to create a suitable prompt for translation
prompt_template = (
    """
    Translate the following text from {source_language} into {target_language}.
    {source_language}: {source_text}
    {target_language}:
    """
)

### C. Perform real-time inference

In [ ]:
# Set whether you'd like a streaming output (or the whole response at once)
stream_output = True

predictor = Predictor(
    endpoint_name=model_name,
    sagemaker_session=session,
    serializer=serializers.JSONSerializer(),
    deserializer=deserializers.JSONLinesDeserializer() if stream_output else deserializers.JSONDeserializer(),
)

In [ ]:
outputs = (
    predictor.predict(
        {
            "messages": [
                {"role": "user", "content": prompt_template.format(**sample)},
            ],
            "max_tokens": 256,
            "temperature": 0.0,
            "stream": stream_output,
            # Check available parameters here: https://docs.djl.ai/master/docs/serving/serving/docs/lmi/user_guides/chat_input_output_schema.html#request-schema
        }
    )
    for sample in inputs
)

### D. Visualize output

In [ ]:
if stream_output:
    # Show completion chunks as they come
    for output in outputs:
        for chunk in output:
            print(chunk["choices"][0]["delta"]["content"], end="")
        print()
else:
    for output in outputs:
        # output is a dictionary with the final response
        print(output["choices"][0]["message"]["content"])

### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint anymore. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Clean-up

### A. Delete the model

In [ ]:
model.delete_model()

### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

